# Getting started with ETL

**ETL** stands for **Extract, Transform, Load**. It is a data integration process used to collect data from multiple sources, clean and transform the data into a proper format, and load it into a target system, such as a data warehouse or database.

## ETL Process Breakdown

1. **Extract**:
   The first step involves extracting data from various sources, such as databases, APIs, or files (e.g., CSV, JSON). The goal is to retrieve the raw data needed for analysis.

2. **Transform**:
   In this stage, the extracted data is cleaned, formatted, and transformed to ensure consistency and usability. Transformations might include:
   - Handling missing or null values
   - Standardizing data formats (e.g., date formats, currencies)
   - Aggregating data or performing calculations
   - Merging data from multiple sources

3. **Load**:
   The final step is loading the transformed data into a target system, such as a data warehouse (e.g., Amazon Redshift, Google BigQuery) or database. This makes the data ready for reporting, analytics, or use by machine learning models.

## Why ETL?

ETL automates the process of collecting, processing, and storing data, ensuring that clean and consistent data is always available for decision-making. This process is essential for businesses that need to handle large volumes of data from multiple sources efficiently and consistently.

## Example

Imagine a retail company that wants to combine data from its online store, physical store, and customer service platform. ETL would:
1. **Extract** sales data from the online and physical store databases and customer feedback from the service platform.
2. **Transform** the data to standardize currencies, clean any missing values, and ensure all timestamps use the same format.
3. **Load** the consolidated data into a centralized database, ready for analysis.

By automating ETL, the company ensures that its data is always up-to-date and ready for reporting and analytics.

---

# First Impressions

In the code below is shown a simple example of how to connect with the google-cloud bigquery and extract data from a table.

In [ ]:
!pip install google-cloud-bigquery
!pip install --upgrade google-cloud-bigquery
!pip install pandas_gbq
!pip install pandas
!pip install load_dotenv
!pip install openpyxl
!pip install google-cloud-bigquery-storage


## Step 1: Install the BigQuery Library

In [5]:
from google.cloud import bigquery
import pandas as pd
from load_dotenv import load_dotenv
import os

## Step 2: Authenticate with Google Cloud and Get the .env file

In [6]:
load_dotenv()

FIRST_PROJECT = os.getenv('FIRST_PROJECT')

credential = 'key.json'
client = bigquery.Client.from_service_account_json(credential)

## Step 3: Load Data from BigQuery

In [7]:
query = f"""
SELECT * FROM `{FIRST_PROJECT}`
"""

result = client.query(query)
df = result.to_dataframe()

df.head()

,id,created_at,first_name,last_name,email,cell_phone,country,state,street,number,additionals
0,36,2018-01-26 03:29:13+00:00,Mariana,Góes,mariana@meu_email.com,9 7324-4293,Brasil,None,None,NaN,Apto 25
1,73,2018-02-03 07:37:38+00:00,Cristiano,Almeida,cristiano@usuario.com,9 2630-9907,Brasil,None,None,NaN,Conjunto 24
2,84,2018-11-01 15:39:40+00:00,Carol,Bueno,carol@meu_email.com,9 3760-2211,Brasil,None,None,NaN,Conjunto 26
3,95,2018-10-01 11:35:59+00:00,Mariana,Rosa,mariana@usuario.com,9 3139-2145,Brasil,None,None,NaN,None
4,0,2017-11-01 14:45:41+00:00,Marta,Jesus,None,9 9102-7834,Brasil,Acre,None,NaN,Conjunto 16


---

## How to transfer data locally to BigQuery

In [8]:
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas_gbq as pd_gbq

In [9]:
credentials = service_account.Credentials.from_service_account_file('key.json')

In [10]:
df = pd.read_excel('data/Produto.xlsx')

In [11]:
df.head()

,ID,Name,Price,Id_Category
0,0,Sapato vermelho,96,0
1,1,Bolsa mais que Velha,8000,0
2,2,Calça,75,0
3,3,Camisa apertadinha,170,0
4,4,Vestido Super curto,57,0


In [12]:
load_dotenv()
SEND = os.getenv('SEND_DATA')
PROJECT_ID = os.getenv('PROJECT_ID')

In [13]:
pd_gbq.to_gbq(
    df,
    destination_table=SEND,
    project_id = PROJECT_ID,
    if_exists = 'replace',
    credentials=credentials
)

---

# Dealing with more than one table

In [14]:
products = pd.read_excel('data/Produto.xlsx')
itens = pd.read_excel('data/items.xlsx')
order = pd.read_excel('data/Ordens.xlsx')
category = pd.read_excel('data/Categoria.xlsx')
clients = pd.read_csv('data/Clientes.csv')

In [15]:
print(f'Products:\n {products.head(2)}\n\n Itens:\n {itens.head(2)}\n\n Order:\n {order.head(2)}\n\n Category:\n {category.head(2)}\n\n Clients:\n {clients.head(2)}')

Products:
    ID                  Name  Price  Id_Category
0   0       Sapato vermelho     96            0
1   1  Bolsa mais que Velha   8000            0

 Itens:
    id  order_id  product_id  quantity  total_price
0   0      7019          28         1          186
1   1      8827          29         1          110

 Order:
    id          created_at  customer_id    status
0   0 2019-08-28 03:36:31           91  entregue
1   1 2019-08-16 20:21:30            7  entregue

 Category:
    id                     name
0   0        Moda e Acessórios
1   1  Cosméticos e Perfumaria

 Clients:
    id           created_at first_name last_name email   cell_phone country  \
0   0  2017-11-01 14:45:41      Marta     Jesus   NaN  9 9102-7834  Brasil   
1   1  2017-10-16 00:50:39      Luana   Almeida   NaN  9 7328-8718  Brasil   

               state                                  street  number  \
0               Acre                                     NaN     NaN   
1  Rio Grande do Sul  Avenid

## Creating a dataset with client connection

In [16]:
load_dotenv()
DATASET_ID = os.getenv('DATASET_ID')

In [17]:
dataset_id = f'{PROJECT_ID}.{DATASET_ID}'
dataset = bigquery.Dataset(dataset_id)

dataset.location = 'US'

dataset = client.create_dataset(dataset, exists_ok=True)

---

In [18]:
credential = service_account.Credentials.from_service_account_file('key.json')

In [19]:
table_names = ['products', 'itens', 'order', 'category', 'clients']

for table, name in zip([products, itens, order, category, clients], table_names):
    pd_gbq.to_gbq(
        table,
        f'{PROJECT_ID}.{DATASET_ID}.{name}',
        project_id=PROJECT_ID,
        if_exists='replace',
        credentials=credential
    )


---

# Pulling data from multiple tables

In [20]:
from google.cloud import bigquery
import pandas

In [21]:
credential = 'key.json'
client = bigquery.Client.from_service_account_json(credential)

In [22]:
tables = ['products', 'itens', 'order', 'category', 'clients']
dfs = {}

In [23]:
for table in tables:
    query = f"""
    SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.{table}`
    """
    result = client.query(query)
    df = result.to_dataframe()
    dfs[table] = df

In [24]:
dfs.keys()

dict_keys(['products', 'itens', 'order', 'category', 'clients'])

In [25]:
dfs['products'].head()

,ID,Name,Price,Id_Category
0,0,Sapato vermelho,96,0
1,1,Bolsa mais que Velha,8000,0
2,2,Calça,75,0
3,3,Camisa apertadinha,170,0
4,4,Vestido Super curto,57,0


In [26]:
dfs['order'].head()

,id,created_at,customer_id,status
0,363,2019-10-30 11:47:36+00:00,0,carrinho
1,489,2019-02-20 04:18:16+00:00,0,carrinho
2,1799,2019-07-20 09:39:59+00:00,0,carrinho
3,2068,2019-09-08 05:02:58+00:00,0,carrinho
4,2405,2019-06-17 22:27:25+00:00,0,carrinho


In [27]:
dfs['order']['created_at'] = dfs['order']['created_at'].dt.tz_convert(None)

In [28]:
dfs['order'].head()

,id,created_at,customer_id,status
0,363,2019-10-30 11:47:36,0,carrinho
1,489,2019-02-20 04:18:16,0,carrinho
2,1799,2019-07-20 09:39:59,0,carrinho
3,2068,2019-09-08 05:02:58,0,carrinho
4,2405,2019-06-17 22:27:25,0,carrinho


# Filtering data

In [73]:
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas_gbq as pd_gbq
from datetime import datetime

In [74]:
df = pd.read_excel('data/Ordens.xlsx')

In [75]:
print(df.dtypes)

id                      int64
created_at     datetime64[ns]
customer_id             int64
status                 object
dtype: object


In [76]:
df['created_at'] = pd.to_datetime(df['created_at']).dt.tz_localize(None)

# ns

df['created_at'] = df['created_at'].dt.strftime('%Y-%m-%d')
df['created_at'] = pd.to_datetime(df['created_at'])

In [77]:
df

,id,created_at,customer_id,status
0,0,2019-08-28,91,entregue
1,1,2019-08-16,7,entregue
2,2,2019-07-11,26,pagamento_pendente
3,3,2019-08-27,92,pagamento_pendente
4,4,2019-11-03,63,entrega_pendente
...,...,...,...,...
9995,9995,2020-01-28,0,entrega_pendente
9996,9996,2019-05-08,58,carrinho
9997,9997,2019-04-08,6,entregue
9998,9998,2019-04-18,50,pagamento_pendente


In [88]:
today = datetime.now()

current_month = today.month
current_year = today.year - 5

print(f'Current Month: {current_month}\nCurrent Year: {current_year}')



Current Month: 9
Current Year: 2019


In [89]:
df[df['created_at'].dt.year == current_year]

,id,created_at,customer_id,status
0,0,2019-08-28,91,entregue
1,1,2019-08-16,7,entregue
2,2,2019-07-11,26,pagamento_pendente
3,3,2019-08-27,92,pagamento_pendente
4,4,2019-11-03,63,entrega_pendente
...,...,...,...,...
9993,9993,2019-12-15,12,carrinho
9994,9994,2019-07-31,79,carrinho
9996,9996,2019-05-08,58,carrinho
9997,9997,2019-04-08,6,entregue


In [90]:
df[df['created_at'].dt.month == current_month]

,id,created_at,customer_id,status
18,18,2019-09-04,97,entregue
30,30,2019-09-15,30,entregue
52,52,2019-09-01,44,carrinho
58,58,2019-09-25,98,entrega_pendente
67,67,2019-09-22,29,entregue
...,...,...,...,...
9935,9935,2019-09-05,34,entregue
9947,9947,2019-09-14,35,carrinho
9969,9969,2019-09-15,84,carrinho
9975,9975,2019-09-25,88,entregue


In [91]:
df_filtered = df[(df['created_at'].dt.month == current_month) & (df['created_at'].dt.year == current_year)]
df_filtered.head()

,id,created_at,customer_id,status
18,18,2019-09-04,97,entregue
30,30,2019-09-15,30,entregue
52,52,2019-09-01,44,carrinho
58,58,2019-09-25,98,entrega_pendente
67,67,2019-09-22,29,entregue


In [95]:
credential = bigquery.Client.from_service_account_json('key.json')

In [93]:
pd_gbq.to_gbq(
    df_filtered,
    f'{PROJECT_ID}.{DATASET_ID}.order_filtered',
    project_id=PROJECT_ID,
    if_exists='replace',
    credentials=credential
)


In [96]:
sql = f"""
DELETE FROM `{PROJECT_ID}.{DATASET_ID}.order`
WHERE created_at < '{current_year}-{current_month}-01'
"""

credential.query(sql)

QueryJob<project=big-query-435107, location=US, id=0d26513f-6612-4392-97ed-2e21136eb664>

# How to keep historic

In [97]:
df = pd.read_excel('data/Produto.xlsx')

In [98]:
df['time_extract'] = datetime.now()

In [105]:
df['time_extract'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [106]:
df.head()

,ID,Name,Price,Id_Category,time_extract
0,0,Sapato vermelho,96,0,2024-09-17 05:13:39
1,1,Bolsa mais que Velha,8000,0,2024-09-17 05:13:39
2,2,Calça,75,0,2024-09-17 05:13:39
3,3,Camisa apertadinha,170,0,2024-09-17 05:13:39
4,4,Vestido Super curto,57,0,2024-09-17 05:13:39


In [107]:
df['date_extract'] = datetime.now().date()

In [108]:
df.head()

,ID,Name,Price,Id_Category,time_extract,date_extract
0,0,Sapato vermelho,96,0,2024-09-17 05:13:39,2024-09-17
1,1,Bolsa mais que Velha,8000,0,2024-09-17 05:13:39,2024-09-17
2,2,Calça,75,0,2024-09-17 05:13:39,2024-09-17
3,3,Camisa apertadinha,170,0,2024-09-17 05:13:39,2024-09-17
4,4,Vestido Super curto,57,0,2024-09-17 05:13:39,2024-09-17


In [109]:
df['extract_time_PTBR'] = datetime.now().strftime('%d/%m/%Y')

In [110]:
df.head()

,ID,Name,Price,Id_Category,time_extract,date_extract,extract_time_PTBR
0,0,Sapato vermelho,96,0,2024-09-17 05:13:39,2024-09-17,17/09/2024
1,1,Bolsa mais que Velha,8000,0,2024-09-17 05:13:39,2024-09-17,17/09/2024
2,2,Calça,75,0,2024-09-17 05:13:39,2024-09-17,17/09/2024
3,3,Camisa apertadinha,170,0,2024-09-17 05:13:39,2024-09-17,17/09/2024
4,4,Vestido Super curto,57,0,2024-09-17 05:13:39,2024-09-17,17/09/2024


In [111]:
df['time_extract_PTBR'] = datetime.now().strftime('%H:%M:%S')

In [112]:
df.head()

,ID,Name,Price,Id_Category,time_extract,date_extract,extract_time_PTBR,time_extract_PTBR
0,0,Sapato vermelho,96,0,2024-09-17 05:13:39,2024-09-17,17/09/2024,05:15:23
1,1,Bolsa mais que Velha,8000,0,2024-09-17 05:13:39,2024-09-17,17/09/2024,05:15:23
2,2,Calça,75,0,2024-09-17 05:13:39,2024-09-17,17/09/2024,05:15:23
3,3,Camisa apertadinha,170,0,2024-09-17 05:13:39,2024-09-17,17/09/2024,05:15:23
4,4,Vestido Super curto,57,0,2024-09-17 05:13:39,2024-09-17,17/09/2024,05:15:23


In [113]:
df['time_extract'] = pd.to_datetime(df['time_extract'])
df['date_extract'] = pd.to_datetime(df['date_extract'])

In [114]:
credential = service_account.Credentials.from_service_account_file('key.json')

pd_gbq.to_gbq(
    df,
    f'{PROJECT_ID}.{DATASET_ID}.products',
    project_id=PROJECT_ID,
    if_exists='replace',
    credentials=credential
)
